In [1]:
# Warning: Execute this cell only once for the kernel. Reset the kernel if you need changes.
import ROOT as R
import numpy as np
%jsroot on
import sys
sys.path.append("/data/HPS/hps-analysis/Python")
from root_helpers import SetStyle
R.EnableImplicitMT()                      # EnableImplicitMT must be called before the RDataFrame is constructed.


Welcome to JupyROOT 6.26/10


In [2]:
import os
recompile = True
try:
    if os.path.getmtime('Particles_C.so') - os.path.getmtime('Particles.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("Particles.C++")
else:
    R.gROOT.LoadMacro("Particles_C.so")
R.Particles()

Recompile is not needed


'Particles analysis. V1.0.1 \n'

In [3]:
# Note: runs 14624 through 14673 are with Ebeam = 1.92 GeV. The rest is at 3.742 GeV
Ebeam = 3.742
ch = R.TChain("MiniDST")
ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
#ch.Add("/data/HPS/data/physrun2021/fee/*.root")
print(f"Loaded {ch.GetEntries():,d} events ({ch.GetEntries():7.2e})")
df = R.RDataFrame(ch)

Loaded 36,587,663 events (3.66e+07)


In [4]:
print("Available data names in Tuple:")
ColumnNames=df.GetColumnNames()
ll = 0
pr_colnames = [x for x in ColumnNames if 'ecal' in str(x)]
for nn in pr_colnames:
    if ll < len(nn):
        ll = len(nn)
for n in range(len(pr_colnames)):
    if n%4 == 0:
        print("")
    print(f"{str(pr_colnames[n]):{ll}s}",end="")

Available data names in Tuple:

ecal_cluster_energy     ecal_cluster_hits       ecal_cluster_nhits      ecal_cluster_seed_energy
ecal_cluster_seed_index ecal_cluster_seed_ix    ecal_cluster_seed_iy    ecal_cluster_time       
ecal_cluster_x          ecal_cluster_y          ecal_cluster_z          ecal_hit_energy         
ecal_hit_index_x        ecal_hit_index_y        ecal_hit_time           part_ecal_cluster       
track_x_at_ecal         track_y_at_ecal         track_z_at_ecal         v0_em_pos_ecal_x        
v0_em_pos_ecal_y        v0_ep_pos_ecal_x        v0_ep_pos_ecal_y        

In [5]:
# Declare a local copy of the fiducial cut found in the Particle.C library, so we can easily manipulate the borders used.
R.gInterpreter.Declare("""
RVec<bool> fid_cut(RVec<int> ix, RVec<int> iy){
    RVec<bool> out;
    for(size_t i=0;i< ix.size();++i){
        if( ix[i]>=-16 && ix[i]<=-2 &&
          (
            //( iy[i]>=2  && iy[i]<=4  ) //
            ( iy[i]>=-3 && iy[i]<=-3 )
          ))
        {
            out.push_back(true);
        }else{
            out.push_back(false);
        }
    }
    return out;
}
""")
dfx = df.Define("n_clus","ecal_cluster_energy.size()")\
        .Define("cluster_is_fiducial","fid_cut(ecal_cluster_seed_ix,ecal_cluster_seed_iy)")\
        .Define("ecs_ix_f","RVec<int> out;for(size_t i=0; i<cluster_is_fiducial.size();++i){ if(cluster_is_fiducial[i]) out.push_back(ecal_cluster_seed_ix[i]);}; return out;").Define("ecs_iy_f","RVec<int> out;for(size_t i=0; i<cluster_is_fiducial.size();++i){ if(cluster_is_fiducial[i]) out.push_back(ecal_cluster_seed_iy[i]);}; return out;")\
        .Define("ec_e_f","""
        RVec<double> out;
        for(size_t i=0; i<cluster_is_fiducial.size();++i){
            if(cluster_is_fiducial[i]){out.push_back(ecal_cluster_energy[i]);}
        }
        return out;""")

#
# Calculate the ecal cluster energy for clusters with a fiducial cut (limiting the region in the ECal), and a cut on the
# number of hits in the cluster.

dfx = dfx.Define("ec_e_f_np","""
    RVec<RVec<double>> out;
    for(int n=0;n<15;n++){
        RVec<double> ntmp;
        for(size_t i=0; i<cluster_is_fiducial.size();++i){
            if( cluster_is_fiducial[i] && ecal_cluster_nhits[i] == n) ntmp.push_back(ecal_cluster_energy[i]);
        }
       out.push_back(ntmp);
    }
    return out;""")


df_1 = dfx.Filter("n_clus == 1")

In [6]:
h_ecal_loc = dfx.Histo2D(("h_ecal_loc","Ecal cluster seed (ix,iy)",51,-25.5,25.5,13,-6.5,6.5),"ecal_cluster_seed_ix","ecal_cluster_seed_iy")
h_ecal_loc_fid = dfx.Histo2D(("h_ecal_loc","Ecal cluster seed (ix,iy)",51,-25.5,25.5,13,-6.5,6.5),"ecs_ix_f","ecs_iy_f")

In [7]:
# moved

In [8]:
n_clus = dfx.Histo1D(("n_clus", "Number of clusters", 11, -0.5, 10.5), "n_clus")
h_ecal_cluster_energy  = dfx.Histo1D(("h_ecal_cluster_energy", "Ecal Cluster Energy;E [GeV]", 300, 0., 5.), "ecal_cluster_energy")
h_ecal_cluster_energy_fid  = dfx.Histo1D(("h_ecal_cluster_energy_fid", "Ecal Cluster Energy;E [GeV]", 300, 0., 5.), "ec_e_f")
n_clus_1 = df_1.Histo1D(("n_clus_1","Number of clusters",11,-0.5,10.5),"n_clus")
h_ecal_cluster_energy_1 = df_1.Histo1D(("h_ecal_cluster_energy_1","Ecal Cluster Energy;E [GeV]",300,0.,5.), "ecal_cluster_energy")
h_ecal_cluster_energy_1_fid = df_1.Histo1D(("h_ecal_cluster_energy_1_fid","Ecal Cluster Energy;E [GeV]",300,0.,5.), "ec_e_f")

In [9]:
h_ecal_cluster_e_fid_np = []
for n in range(15):
    ecal_clus_e_with_cut = f"ec_e_f_np[{n}]"
    hist_name = f"h_ecal_cluster_e_fid_np{n}"
    h_ecal_cluster_e_fid_np.append(dfx.Define("ec_e",ecal_clus_e_with_cut).Histo1D((hist_name, "Ecal Cluster Energy;E [GeV]", 300, 0., 5.), "ec_e"))


In [10]:
c0 = R.TCanvas("c0","Canvas1",1000,900)
c0.Divide(1,2)
p1 = c0.cd(1)
p1.SetLogz(1)
h_ecal_loc.SetStats(0)
h_ecal_loc.Draw("colz")
p2 = c0.cd(2)
p2.SetLogz(1)
h_ecal_loc_fid.SetStats(0)
h_ecal_loc_fid.Draw("colz")
c0.Draw()

In [11]:
c1 = R.TCanvas("c1","Canvas1",700,700)
c1.SetLogy(0)
h_ecal_cluster_energy.Draw()
h_ecal_cluster_energy_1.SetLineColor(R.kRed)
h_ecal_cluster_energy_1.Draw("same")
h_ecal_cluster_energy_fid.SetLineColor(R.kGreen)
h_ecal_cluster_energy_fid.Draw("same")
h_ecal_cluster_energy_1_fid.SetLineColor(R.kOrange)
h_ecal_cluster_energy_1_fid.Draw("same")
c1.Draw()

In [12]:
c2 = R.TCanvas("c2","Canvas1",700,700)
c2.SetLogy(1)
line_colors = [R.kBlack,   R.kBlue,   R.kGreen,   R.kOrange,   R.kRed,   R.kTeal,   R.kViolet,
               R.kGray,    R.kBlue+1, R.kGreen+1, R.kOrange+1, R.kRed+1, R.kTeal+1, R.kViolet+1,
               R.kGray+1,  R.kBlue+2, R.kGreen+2, R.kOrange+2, R.kRed+2, R.kTeal+2, R.kViolet+2]
legend = R.TLegend(0.75,0.6,0.9,0.9)
for i in range(9,min(20,len(h_ecal_cluster_e_fid_np))):
    h_ecal_cluster_e_fid_np[i].SetStats(0)
    h_ecal_cluster_e_fid_np[i].SetLineColor(line_colors[i])
    if i==0:
        h_ecal_cluster_e_fid_np[i].Draw()
    else:
        h_ecal_cluster_e_fid_np[i].Draw("same")
    legend.AddEntry(h_ecal_cluster_e_fid_np[i].GetValue(), f"n=={i}")
legend.Draw()
c2.Draw()